In [1]:
# # to delete folders
# !rm -r /content/real_audios
# !rm -r /content/fake_audios
# !rm -r /content/real_images
# !rm -r /content/fake_images

Install Required Libraries

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Authenticate and Set Up Google Drive API

In [3]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.errors import HttpError
import io

# Set up the Google Drive API
def setup_drive_service():
    from googleapiclient.discovery import build
    from google.auth import default

    creds, _ = default()
    service = build('drive', 'v3', credentials=creds)
    return service

drive_service = setup_drive_service()

In [4]:
import os

def list_files_in_folder(folder_id):
    query = f"'{folder_id}' in parents and mimeType != 'application/vnd.google-apps.folder'"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    return results.get('files', [])

def download_file(file_id, destination):
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.FileIO(destination, mode='wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print(f"Download {int(status.progress() * 100)}%.")

def download_files_from_folder(folder_id, destination_folder):
    files = list_files_in_folder(folder_id)
    for file in files:
        file_id = file['id']
        file_name = file['name']
        destination = os.path.join(destination_folder, file_name)
        print(f"Downloading {file_name}...")
        download_file(file_id, destination)
        print(f"Saved to {destination}")

In [ ]:
# Real audio files folder
folder_id_real = '1G-m8xRyHEIhsjWydgUObdjsECPfpU_vP'
destination_folder_real = '/content/real_audios'

# Fake audio files folder
folder_id_fake = '1b-c9h0y0SE2hbsF5gDIs07ykkdMdZVLQ'
destination_folder_fake = '/content/fake_audios'

# Create destination folder if it doesn't exist
os.makedirs(destination_folder_real, exist_ok=True)
os.makedirs(destination_folder_fake, exist_ok=True)

#Downloading files seperately
download_files_from_folder(folder_id_real, destination_folder_real)
download_files_from_folder(folder_id_fake, destination_folder_fake)

Audio to Melspectrogram

In [6]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os

def wav_to_melspectrogram(input_folder, output_folder, n_mels=128, fmax=8000):
    # Creating an output folder
    os.makedirs(output_folder, exist_ok=True)

    print(f"Generating Mel-spectrogram with the following parameters:")
    print(f"Number of Mel bands (n_mels): {n_mels}")
    print(f"Maximum frequency (fmax): {fmax}")

    # Iterating all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.wav'):
            wav_file = os.path.join(input_folder, filename)
            output_image = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.png")

            # Loading the audio file
            y, sr = librosa.load(wav_file, sr=None)
            print(f"\nProcessing file: {filename}")
            print(f"Sample rate (sr): {sr}")

            # Generating the Mel spectrogram
            S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, fmax=fmax)

            # Converting the power spectrogram to dB (log scale)
            S_dB = librosa.power_to_db(S, ref=np.max)

            # Plotting the Mel-spectrogram image
            plt.figure(figsize=(10, 4))
            librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel', fmax=fmax)
            plt.colorbar(format='%+2.0f dB')
            plt.title(f'Mel-frequency spectrogram: {filename}')
            plt.tight_layout()
            plt.savefig(output_image)
            plt.close()
            print(f"Saved {output_image}")

#Prinitng the mel filterbank

    # Generating Mel filter bank
    mel_filterbank = librosa.filters.mel(sr=16000, n_fft=2048 , n_mels=128, fmax=8000)

    # Ploting each filter as a triangular line plot
    plt.figure(figsize=(10, 6))
    for i in range(n_mels):
      plt.plot(mel_filterbank[i], label=f'Mel filter {i+1}', alpha=0.5)

    plt.xlabel('FFT bins')
    plt.ylabel('Amplitude')
    plt.title('Mel Filter Bank - Triangular Filter Shapes')
    plt.show()


Compute Melspectrogram and save images to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
input_folder_real = '/content/real_audios'
output_folder_real = '/content/drive/MyDrive/SampleRun/Computed Images/Melspectrogram/real'

wav_to_melspectrogram(input_folder_real, output_folder_real)

input_folder_fake = '/content/fake_audios'
output_folder_fake = '/content/drive/MyDrive/SampleRun/Computed Images/Melspectrogram/fake'

wav_to_melspectrogram(input_folder_fake, output_folder_fake)